In [ ]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "80%"

In [ ]:
#Inputs
parcels = r".\Outputs\remm_base_year_2020.gdb\parcels"


hui = r"E:\Projects\Housing-Unit-Inventory\Current_Version\wfrc_hui_20211110.gdb\wfrc_hui_20211101"


if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "test.gdb"]
gdb = os.path.join(outputs[0], outputs[1])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

In [ ]:
# buffer hui
hui_buff = arcpy.Buffer_analysis(hui, os.path.join(gdb,'hui_buff'), '1.25 METERS')

In [ ]:
# spatial join 
sj = arcpy.analysis.SpatialJoin(target_features=parcels, 
                           join_features=hui_buff, 
                           out_feature_class=os.path.join(gdb,'sj'), 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_COMMON",
                           match_option="WITHIN")



In [ ]:
# get list of unit ids that joined
sj_sdf = pd.DataFrame.spatial.from_featureclass(sj)
unit_ids = sj_sdf['UNIT_ID'].tolist()  
len(unit_ids)

In [ ]:
hui

In [ ]:
# use ids to identify hui units that didn't join
hui_sdf = pd.DataFrame.spatial.from_featureclass(hui)
not_joined = hui_sdf[~(hui_sdf['UNIT_ID'].isin(unit_ids))].copy()
print(hui_sdf.shape)
print(not_joined.shape)

In [ ]:
# export
not_joined.spatial.to_featureclass(location=os.path.join(gdb,'unjoined_hui_parcels'),sanitize_columns=False)